In [79]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm

import scipy
import pylab
import math
import random

import sklearn
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler, RobustScaler
from sklearn.neighbors import NearestNeighbors
from pandas.api.types import is_numeric_dtype
from fuzzywuzzy import fuzz
from sklearn.metrics.pairwise import haversine_distances

%matplotlib inline
sns.set()

In [2]:
df = pd.read_excel('Выборка_ектб.xlsx', sheet_name = 0)
df

,ID,Мусор,Агрегатор,LN(Цена),Цена,Цена_перв,Прогноз_1,Прогноз_2,MAE,MAPE,...,Нежилые помещения на первом этаже,строение,Квартира угловая / торцевая,Способ продажи,Отделка,Название новостройки,"Корпус, строение",Тип участия,Срок сдачи,Запланирован снос
0,1_этажи,0,этажи,15.761421,7000000,7200000,7.357429e+06,7.315261e+06,-315260.780670,-0.045037,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2_этажи,0,этажи,16.231424,11200000,11400000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3_этажи,0,этажи,15.274126,4300000,4500000,4.955072e+06,4.923574e+06,-623574.436842,-0.145017,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4_этажи,0,этажи,15.555977,5700000,6000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5_этажи,0,этажи,14.557448,2100000,2200000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13331,8466_авито,1,авито,15.659230,6320000,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,переуступка,NaN,ЖК «Ботаника LIFE»,NaN,другое,Сдан 4 кв. 2023,NaN
13332,8467_авито,0,авито,15.147877,3790000,0,NaN,4.253817e+06,-463816.720736,-0.122379,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13333,8468_авито,0,авито,15.054464,3452000,0,NaN,3.777030e+06,-325030.387942,-0.094157,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13334,8469_авито,0,авито,15.176487,3900000,0,NaN,3.866094e+06,33905.795052,0.008694,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df = df[df['Мусор'] == 0]
df = df[df['Выбросы'] == 0]
df = df[df['dup_res'] == 0]
df = df[df['Многоуровневость'] == 0]
# df = df[df['Терраса'] == 0]
df = df[df['Вид_1'] == 'нет']
df.reset_index(drop = True , inplace = True)
df['Адрес_yandex'] = df['Адрес_yandex'].str.replace(r'Россия, Свердловская область, ', '', regex = True) # Сокращение адреса
df

,ID,Мусор,Агрегатор,LN(Цена),Цена,Цена_перв,Прогноз_1,Прогноз_2,MAE,MAPE,...,Нежилые помещения на первом этаже,строение,Квартира угловая / торцевая,Способ продажи,Отделка,Название новостройки,"Корпус, строение",Тип участия,Срок сдачи,Запланирован снос
0,1_этажи,0,этажи,15.761421,7000000,7200000,7.357429e+06,7.315261e+06,-315260.780670,-0.045037,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3_этажи,0,этажи,15.274126,4300000,4500000,4.955072e+06,4.923574e+06,-623574.436842,-0.145017,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6_этажи,0,этажи,15.483217,5300000,0,6.115291e+06,6.056768e+06,-756768.110306,-0.142786,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7_этажи,0,этажи,15.679591,6450000,6490000,6.246966e+06,6.173033e+06,276967.222331,0.042941,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,15_этажи,0,этажи,15.823725,7450000,7500000,7.502204e+06,7.472315e+06,-22315.213466,-0.002995,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7884,8464_авито,0,авито,15.293700,4385000,0,5.291969e+06,5.297718e+06,-912718.329665,-0.208146,...,NaN,NaN,NaN,NaN,NaN,ЖК «Миллениум»,"Дом 5 (секц. А, Б)",другое,Сдан 2 кв. 2017,NaN
7885,8467_авито,0,авито,15.147877,3790000,0,NaN,4.253817e+06,-463816.720736,-0.122379,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7886,8468_авито,0,авито,15.054464,3452000,0,NaN,3.777030e+06,-325030.387942,-0.094157,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7887,8469_авито,0,авито,15.176487,3900000,0,NaN,3.866094e+06,33905.795052,0.008694,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df_sample = df[[
                'Цена(кв.м)',
                'ОПлощадь',
                'Этаж_1',
                'ГП',
                'Ремонт_1',
                'Районы_2',
                'Сцепка_к',
                'ЖК_БД',
                'Терраса',
                'Адрес_yandex'
              ]]
df_sample

,Цена(кв.м),ОПлощадь,Этаж_1,ГП,Ремонт_1,Районы_2,Сцепка_к,ЖК_БД,Терраса,Адрес_yandex
0,99573.257468,70.30,3,2009.0,стандартный,Академический,60.514731 56.810482,нет,0.0,"Екатеринбург, улица Соболева, 19"
1,106435.643564,40.40,4,1960.0,стандартный,Центральный,60.636515 56.826183,нет,0.0,"Екатеринбург, улица Декабристов, 4"
2,124941.065535,42.42,14,2012.0,стандартный,Пионерский,60.648157 56.864782,нет,0.0,"Екатеринбург, Боровая улица, 31"
3,113556.338028,56.80,4,2014.0,стандартный,Уктус,60.681647 56.782174,нет,0.0,"Екатеринбург, Алтайская улица, 62"
4,168552.036199,44.20,6,2016.0,стандартный,Южный,60.616276 56.804894,нет,0.0,"Екатеринбург, улица Степана Разина, 107Ак2"
...,...,...,...,...,...,...,...,...,...,...
7884,141451.612903,31.00,13,2017.0,стандартный,Юго_Западный,60.572447 56.81096,нет,0.0,"Екатеринбург, улица Громова, 26"
7885,99475.065617,38.10,9,1994.0,стандартный,Уктус,60.649271 56.795727,нет,0.0,"Екатеринбург, Просторная улица, 85"
7886,106543.209877,32.40,9,1994.0,стандартный,Уктус,60.649271 56.795727,нет,0.0,"Екатеринбург, Просторная улица, 85"
7887,118902.439024,32.80,6,1994.0,стандартный,Уктус,60.649271 56.795727,нет,0.0,"Екатеринбург, Просторная улица, 85"


In [5]:
lats = []
lons = []

for i in range(len(df_sample['Сцепка_к'])):
    lat = df_sample['Сцепка_к'][i].split(' ')[0]
    lon = df_sample['Сцепка_к'][i].split(' ')[1]
            
    lats.append(lat)
    lons.append(lon)
    
df_sample['lat'] = [float(i) for i in lats]
df_sample['lon'] = [float(i) for i in lons]

/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [7]:
df_coord = df_sample[['lat', 'lon']]

In [12]:
scaler = MinMaxScaler()
df_coord_scale = scaler.fit_transform(df_coord)
#df_coord_scale

In [13]:
n_neighbors = 1  # Количество ближайших соседей
metric = 'manhattan' # Список метрик для перебора, minkowski, manhattan
algorithm = 'ball_tree' # Список алгоритмов для перебора

knn = NearestNeighbors(
                       n_neighbors = n_neighbors + 1,
                       algorithm = algorithm,
                       metric = metric
                      )

knn.fit(df_coord_scale)

distances, neighbors_indices = knn.kneighbors(df_coord_scale)

In [34]:
df_sample

,Цена(кв.м),ОПлощадь,Этаж_1,ГП,Ремонт_1,Районы_2,Сцепка_к,ЖК_БД,Терраса,Адрес_yandex,lat,lon
0,99573.257468,70.30,3,2009.0,стандартный,Академический,60.514731 56.810482,нет,0.0,"Екатеринбург, улица Соболева, 19",60.514731,56.810482
1,106435.643564,40.40,4,1960.0,стандартный,Центральный,60.636515 56.826183,нет,0.0,"Екатеринбург, улица Декабристов, 4",60.636515,56.826183
2,124941.065535,42.42,14,2012.0,стандартный,Пионерский,60.648157 56.864782,нет,0.0,"Екатеринбург, Боровая улица, 31",60.648157,56.864782
3,113556.338028,56.80,4,2014.0,стандартный,Уктус,60.681647 56.782174,нет,0.0,"Екатеринбург, Алтайская улица, 62",60.681647,56.782174
4,168552.036199,44.20,6,2016.0,стандартный,Южный,60.616276 56.804894,нет,0.0,"Екатеринбург, улица Степана Разина, 107Ак2",60.616276,56.804894
...,...,...,...,...,...,...,...,...,...,...,...,...
7884,141451.612903,31.00,13,2017.0,стандартный,Юго_Западный,60.572447 56.81096,нет,0.0,"Екатеринбург, улица Громова, 26",60.572447,56.810960
7885,99475.065617,38.10,9,1994.0,стандартный,Уктус,60.649271 56.795727,нет,0.0,"Екатеринбург, Просторная улица, 85",60.649271,56.795727
7886,106543.209877,32.40,9,1994.0,стандартный,Уктус,60.649271 56.795727,нет,0.0,"Екатеринбург, Просторная улица, 85",60.649271,56.795727
7887,118902.439024,32.80,6,1994.0,стандартный,Уктус,60.649271 56.795727,нет,0.0,"Екатеринбург, Просторная улица, 85",60.649271,56.795727


In [44]:
#удаление самоподобия
for i, row in enumerate(neighbors_indices):
    new_row = np.delete(row, np.where(row == i))
    neighbors_indices[i] = new_row
    
neighbors_indices

array([[1259, 1259],
       [2950, 2950],
       [3585, 3585],
       ...,
       [7885, 7887],
       [7885, 7885],
       [7885, 7887]])

In [50]:
neighbors_indices = np.delete(neighbors_indices, 1, 1) #номер объекта, тип - строка (0) / столбец (1)
neighbors_indices

array([[1259],
       [2950],
       [3585],
       ...,
       [7885],
       [7885],
       [7885]])

In [51]:
df_sample['OA_ind'] = neighbors_indices
df_sample

/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Цена(кв.м),ОПлощадь,Этаж_1,ГП,Ремонт_1,Районы_2,Сцепка_к,ЖК_БД,Терраса,Адрес_yandex,lat,lon,OA_ind
0,99573.257468,70.30,3,2009.0,стандартный,Академический,60.514731 56.810482,нет,0.0,"Екатеринбург, улица Соболева, 19",60.514731,56.810482,1259
1,106435.643564,40.40,4,1960.0,стандартный,Центральный,60.636515 56.826183,нет,0.0,"Екатеринбург, улица Декабристов, 4",60.636515,56.826183,2950
2,124941.065535,42.42,14,2012.0,стандартный,Пионерский,60.648157 56.864782,нет,0.0,"Екатеринбург, Боровая улица, 31",60.648157,56.864782,3585
3,113556.338028,56.80,4,2014.0,стандартный,Уктус,60.681647 56.782174,нет,0.0,"Екатеринбург, Алтайская улица, 62",60.681647,56.782174,2285
4,168552.036199,44.20,6,2016.0,стандартный,Южный,60.616276 56.804894,нет,0.0,"Екатеринбург, улица Степана Разина, 107Ак2",60.616276,56.804894,1341
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7884,141451.612903,31.00,13,2017.0,стандартный,Юго_Западный,60.572447 56.81096,нет,0.0,"Екатеринбург, улица Громова, 26",60.572447,56.810960,3788
7885,99475.065617,38.10,9,1994.0,стандартный,Уктус,60.649271 56.795727,нет,0.0,"Екатеринбург, Просторная улица, 85",60.649271,56.795727,7887
7886,106543.209877,32.40,9,1994.0,стандартный,Уктус,60.649271 56.795727,нет,0.0,"Екатеринбург, Просторная улица, 85",60.649271,56.795727,7885
7887,118902.439024,32.80,6,1994.0,стандартный,Уктус,60.649271 56.795727,нет,0.0,"Екатеринбург, Просторная улица, 85",60.649271,56.795727,7885


In [56]:
mapping = dict(zip(df_sample.index, df_sample['Адрес_yandex']))
df_sample['OA_add'] = df_sample['OA_ind'].apply(lambda x: mapping.get(x))
df_sample

/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Цена(кв.м),ОПлощадь,Этаж_1,ГП,Ремонт_1,Районы_2,Сцепка_к,ЖК_БД,Терраса,Адрес_yandex,lat,lon,OA_ind,OA_add
0,99573.257468,70.30,3,2009.0,стандартный,Академический,60.514731 56.810482,нет,0.0,"Екатеринбург, улица Соболева, 19",60.514731,56.810482,1259,"Екатеринбург, улица Соболева, 19"
1,106435.643564,40.40,4,1960.0,стандартный,Центральный,60.636515 56.826183,нет,0.0,"Екатеринбург, улица Декабристов, 4",60.636515,56.826183,2950,"Екатеринбург, улица Декабристов, 6"
2,124941.065535,42.42,14,2012.0,стандартный,Пионерский,60.648157 56.864782,нет,0.0,"Екатеринбург, Боровая улица, 31",60.648157,56.864782,3585,"Екатеринбург, Боровая улица, 31"
3,113556.338028,56.80,4,2014.0,стандартный,Уктус,60.681647 56.782174,нет,0.0,"Екатеринбург, Алтайская улица, 62",60.681647,56.782174,2285,"Екатеринбург, Якутская улица, 10"
4,168552.036199,44.20,6,2016.0,стандартный,Южный,60.616276 56.804894,нет,0.0,"Екатеринбург, улица Степана Разина, 107Ак2",60.616276,56.804894,1341,"Екатеринбург, улица Степана Разина, 107Ак2"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7884,141451.612903,31.00,13,2017.0,стандартный,Юго_Западный,60.572447 56.81096,нет,0.0,"Екатеринбург, улица Громова, 26",60.572447,56.810960,3788,"Екатеринбург, улица Громова, 26"
7885,99475.065617,38.10,9,1994.0,стандартный,Уктус,60.649271 56.795727,нет,0.0,"Екатеринбург, Просторная улица, 85",60.649271,56.795727,7887,"Екатеринбург, Просторная улица, 85"
7886,106543.209877,32.40,9,1994.0,стандартный,Уктус,60.649271 56.795727,нет,0.0,"Екатеринбург, Просторная улица, 85",60.649271,56.795727,7885,"Екатеринбург, Просторная улица, 85"
7887,118902.439024,32.80,6,1994.0,стандартный,Уктус,60.649271 56.795727,нет,0.0,"Екатеринбург, Просторная улица, 85",60.649271,56.795727,7885,"Екатеринбург, Просторная улица, 85"


In [63]:
sim = []

for i in tqdm(range(len(df_sample['Адрес_yandex'])), total = df_sample.shape[0], position = 0):
    token_set_ratio = fuzz.token_set_ratio(df_sample['Адрес_yandex'][i], df_sample['OA_add'][i])
    sim.append(token_set_ratio)

df_sample['sim'] = sim

100%|██████████| 7889/7889 [00:01<00:00, 7204.89it/s]
/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [64]:
df_sample

,Цена(кв.м),ОПлощадь,Этаж_1,ГП,Ремонт_1,Районы_2,Сцепка_к,ЖК_БД,Терраса,Адрес_yandex,lat,lon,OA_ind,OA_add,sim
0,99573.257468,70.30,3,2009.0,стандартный,Академический,60.514731 56.810482,нет,0.0,"Екатеринбург, улица Соболева, 19",60.514731,56.810482,1259,"Екатеринбург, улица Соболева, 19",100
1,106435.643564,40.40,4,1960.0,стандартный,Центральный,60.636515 56.826183,нет,0.0,"Екатеринбург, улица Декабристов, 4",60.636515,56.826183,2950,"Екатеринбург, улица Декабристов, 6",97
2,124941.065535,42.42,14,2012.0,стандартный,Пионерский,60.648157 56.864782,нет,0.0,"Екатеринбург, Боровая улица, 31",60.648157,56.864782,3585,"Екатеринбург, Боровая улица, 31",100
3,113556.338028,56.80,4,2014.0,стандартный,Уктус,60.681647 56.782174,нет,0.0,"Екатеринбург, Алтайская улица, 62",60.681647,56.782174,2285,"Екатеринбург, Якутская улица, 10",82
4,168552.036199,44.20,6,2016.0,стандартный,Южный,60.616276 56.804894,нет,0.0,"Екатеринбург, улица Степана Разина, 107Ак2",60.616276,56.804894,1341,"Екатеринбург, улица Степана Разина, 107Ак2",100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7884,141451.612903,31.00,13,2017.0,стандартный,Юго_Западный,60.572447 56.81096,нет,0.0,"Екатеринбург, улица Громова, 26",60.572447,56.810960,3788,"Екатеринбург, улица Громова, 26",100
7885,99475.065617,38.10,9,1994.0,стандартный,Уктус,60.649271 56.795727,нет,0.0,"Екатеринбург, Просторная улица, 85",60.649271,56.795727,7887,"Екатеринбург, Просторная улица, 85",100
7886,106543.209877,32.40,9,1994.0,стандартный,Уктус,60.649271 56.795727,нет,0.0,"Екатеринбург, Просторная улица, 85",60.649271,56.795727,7885,"Екатеринбург, Просторная улица, 85",100
7887,118902.439024,32.80,6,1994.0,стандартный,Уктус,60.649271 56.795727,нет,0.0,"Екатеринбург, Просторная улица, 85",60.649271,56.795727,7885,"Екатеринбург, Просторная улица, 85",100


In [67]:
address_threshold = 94

df_sample[df_sample['sim'] >= address_threshold]

,Цена(кв.м),ОПлощадь,Этаж_1,ГП,Ремонт_1,Районы_2,Сцепка_к,ЖК_БД,Терраса,Адрес_yandex,lat,lon,OA_ind,OA_add,sim
0,99573.257468,70.30,3,2009.0,стандартный,Академический,60.514731 56.810482,нет,0.0,"Екатеринбург, улица Соболева, 19",60.514731,56.810482,1259,"Екатеринбург, улица Соболева, 19",100
1,106435.643564,40.40,4,1960.0,стандартный,Центральный,60.636515 56.826183,нет,0.0,"Екатеринбург, улица Декабристов, 4",60.636515,56.826183,2950,"Екатеринбург, улица Декабристов, 6",97
2,124941.065535,42.42,14,2012.0,стандартный,Пионерский,60.648157 56.864782,нет,0.0,"Екатеринбург, Боровая улица, 31",60.648157,56.864782,3585,"Екатеринбург, Боровая улица, 31",100
4,168552.036199,44.20,6,2016.0,стандартный,Южный,60.616276 56.804894,нет,0.0,"Екатеринбург, улица Степана Разина, 107Ак2",60.616276,56.804894,1341,"Екатеринбург, улица Степана Разина, 107Ак2",100
5,143312.101911,31.40,4,1958.0,стандартный,Центральный,60.615198 56.853218,нет,0.0,"Екатеринбург, улица Луначарского, 36",60.615198,56.853218,2181,"Екатеринбург, улица Луначарского, 21",95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7884,141451.612903,31.00,13,2017.0,стандартный,Юго_Западный,60.572447 56.81096,нет,0.0,"Екатеринбург, улица Громова, 26",60.572447,56.810960,3788,"Екатеринбург, улица Громова, 26",100
7885,99475.065617,38.10,9,1994.0,стандартный,Уктус,60.649271 56.795727,нет,0.0,"Екатеринбург, Просторная улица, 85",60.649271,56.795727,7887,"Екатеринбург, Просторная улица, 85",100
7886,106543.209877,32.40,9,1994.0,стандартный,Уктус,60.649271 56.795727,нет,0.0,"Екатеринбург, Просторная улица, 85",60.649271,56.795727,7885,"Екатеринбург, Просторная улица, 85",100
7887,118902.439024,32.80,6,1994.0,стандартный,Уктус,60.649271 56.795727,нет,0.0,"Екатеринбург, Просторная улица, 85",60.649271,56.795727,7885,"Екатеринбург, Просторная улица, 85",100


# Оценка в 4 ступени

In [106]:
df = pd.read_excel('Выборка_ектб.xlsx', sheet_name = 0)
df

,ID,Мусор,Агрегатор,LN(Цена),Цена,Цена_перв,Прогноз_1,Прогноз_2,MAE,MAPE,...,Нежилые помещения на первом этаже,строение,Квартира угловая / торцевая,Способ продажи,Отделка,Название новостройки,"Корпус, строение",Тип участия,Срок сдачи,Запланирован снос
0,1_этажи,0,этажи,15.761421,7000000,7200000,7.357429e+06,7.315261e+06,-315260.780670,-0.045037,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2_этажи,0,этажи,16.231424,11200000,11400000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3_этажи,0,этажи,15.274126,4300000,4500000,4.955072e+06,4.923574e+06,-623574.436842,-0.145017,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4_этажи,0,этажи,15.555977,5700000,6000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5_этажи,0,этажи,14.557448,2100000,2200000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13331,8466_авито,1,авито,15.659230,6320000,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,переуступка,NaN,ЖК «Ботаника LIFE»,NaN,другое,Сдан 4 кв. 2023,NaN
13332,8467_авито,0,авито,15.147877,3790000,0,NaN,4.253817e+06,-463816.720736,-0.122379,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13333,8468_авито,0,авито,15.054464,3452000,0,NaN,3.777030e+06,-325030.387942,-0.094157,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13334,8469_авито,0,авито,15.176487,3900000,0,NaN,3.866094e+06,33905.795052,0.008694,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [107]:
df = df[df['Мусор'] == 0]
df = df[df['Выбросы'] == 0]
df = df[df['dup_res'] == 0]
df = df[df['Многоуровневость'] == 0]
# df = df[df['Терраса'] == 0]
df = df[df['Вид_1'] == 'нет']
df.reset_index(drop = True , inplace = True)
df['Адрес_yandex'] = df['Адрес_yandex'].str.replace(r'Россия, Свердловская область, ', '', regex = True) # Сокращение адреса
df

,ID,Мусор,Агрегатор,LN(Цена),Цена,Цена_перв,Прогноз_1,Прогноз_2,MAE,MAPE,...,Нежилые помещения на первом этаже,строение,Квартира угловая / торцевая,Способ продажи,Отделка,Название новостройки,"Корпус, строение",Тип участия,Срок сдачи,Запланирован снос
0,1_этажи,0,этажи,15.761421,7000000,7200000,7.357429e+06,7.315261e+06,-315260.780670,-0.045037,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3_этажи,0,этажи,15.274126,4300000,4500000,4.955072e+06,4.923574e+06,-623574.436842,-0.145017,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6_этажи,0,этажи,15.483217,5300000,0,6.115291e+06,6.056768e+06,-756768.110306,-0.142786,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7_этажи,0,этажи,15.679591,6450000,6490000,6.246966e+06,6.173033e+06,276967.222331,0.042941,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,15_этажи,0,этажи,15.823725,7450000,7500000,7.502204e+06,7.472315e+06,-22315.213466,-0.002995,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7884,8464_авито,0,авито,15.293700,4385000,0,5.291969e+06,5.297718e+06,-912718.329665,-0.208146,...,NaN,NaN,NaN,NaN,NaN,ЖК «Миллениум»,"Дом 5 (секц. А, Б)",другое,Сдан 2 кв. 2017,NaN
7885,8467_авито,0,авито,15.147877,3790000,0,NaN,4.253817e+06,-463816.720736,-0.122379,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7886,8468_авито,0,авито,15.054464,3452000,0,NaN,3.777030e+06,-325030.387942,-0.094157,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7887,8469_авито,0,авито,15.176487,3900000,0,NaN,3.866094e+06,33905.795052,0.008694,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [108]:
df_sample = df[[
                'Цена(кв.м)',
                'ОПлощадь',
                'Этаж_1',
                'ГП',
                'Ремонт_1',
                'Районы_2',
                'Сцепка_к',
                'ЖК_БД',
                'Терраса',
                'Адрес_yandex'
              ]]
df_sample

,Цена(кв.м),ОПлощадь,Этаж_1,ГП,Ремонт_1,Районы_2,Сцепка_к,ЖК_БД,Терраса,Адрес_yandex
0,99573.257468,70.30,3,2009.0,стандартный,Академический,60.514731 56.810482,нет,0.0,"Екатеринбург, улица Соболева, 19"
1,106435.643564,40.40,4,1960.0,стандартный,Центральный,60.636515 56.826183,нет,0.0,"Екатеринбург, улица Декабристов, 4"
2,124941.065535,42.42,14,2012.0,стандартный,Пионерский,60.648157 56.864782,нет,0.0,"Екатеринбург, Боровая улица, 31"
3,113556.338028,56.80,4,2014.0,стандартный,Уктус,60.681647 56.782174,нет,0.0,"Екатеринбург, Алтайская улица, 62"
4,168552.036199,44.20,6,2016.0,стандартный,Южный,60.616276 56.804894,нет,0.0,"Екатеринбург, улица Степана Разина, 107Ак2"
...,...,...,...,...,...,...,...,...,...,...
7884,141451.612903,31.00,13,2017.0,стандартный,Юго_Западный,60.572447 56.81096,нет,0.0,"Екатеринбург, улица Громова, 26"
7885,99475.065617,38.10,9,1994.0,стандартный,Уктус,60.649271 56.795727,нет,0.0,"Екатеринбург, Просторная улица, 85"
7886,106543.209877,32.40,9,1994.0,стандартный,Уктус,60.649271 56.795727,нет,0.0,"Екатеринбург, Просторная улица, 85"
7887,118902.439024,32.80,6,1994.0,стандартный,Уктус,60.649271 56.795727,нет,0.0,"Екатеринбург, Просторная улица, 85"


In [129]:
lats = []
lons = []

for i in range(len(df_sample['Сцепка_к'])):
    lon = df_sample['Сцепка_к'][i].split(' ')[0]
    lat = df_sample['Сцепка_к'][i].split(' ')[1]
            
    lats.append(lat)
    lons.append(lon)
    
df_sample['lat'] = [float(i) for i in lats]
df_sample['lon'] = [float(i) for i in lons]

/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [130]:
df_coord_oo = df_sample[['lat', 'lon']]
df_coord_oo

,lat,lon
0,56.810482,60.514731
1,56.826183,60.636515
2,56.864782,60.648157
3,56.782174,60.681647
4,56.804894,60.616276
...,...,...
7884,56.810960,60.572447
7885,56.795727,60.649271
7886,56.795727,60.649271
7887,56.795727,60.649271


In [131]:
df_elit = pd.read_excel('БД_элитка, виды.xlsx', sheet_name = 0)
df_elit = df_elit[['Широта', 'Долгота', 'Адрес_ya', 'Статус']]
df_elit

,Широта,Долгота,Адрес_ya,Статус
0,56.830408,60.590854,"Россия, Свердловская область, Екатеринбург, ул...",престижный+
1,56.830635,60.592956,"Россия, Свердловская область, Екатеринбург, ул...",престижный+
2,56.841379,60.591402,"Россия, Свердловская область, Екатеринбург, ул...",престижный+
3,56.820163,60.591689,"Россия, Свердловская область, Екатеринбург, ул...",престижный+
4,56.819464,60.591959,"Россия, Свердловская область, Екатеринбург, ул...",престижный+
...,...,...,...,...
97,56.830857,60.616680,"Россия, Свердловская область, Екатеринбург, ул...",престижный
98,56.830359,60.615611,"Россия, Свердловская область, Екатеринбург, ул...",престижный
99,56.830620,60.614560,"Россия, Свердловская область, Екатеринбург, ул...",престижный
100,56.807422,60.614336,"Россия, Свердловская область, Екатеринбург, ул...",престижный


In [132]:
df_coord_oa = df_elit[['Широта', 'Долгота']]
df_coord_oa = df_coord_oa.rename(columns = {'Широта': 'lat', 'Долгота': 'lon'})
df_coord_oa

,lat,lon
0,56.830408,60.590854
1,56.830635,60.592956
2,56.841379,60.591402
3,56.820163,60.591689
4,56.819464,60.591959
...,...,...
97,56.830857,60.616680
98,56.830359,60.615611
99,56.830620,60.614560
100,56.807422,60.614336


In [133]:
combined_data = pd.concat([df_coord_oo, df_coord_oa])

In [134]:
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(combined_data)

In [135]:
table_oo_scaled = scaled_data[:len(df_coord_oo), :]
table_oa_scaled = scaled_data[len(df_coord_oo):, :]

In [136]:
print(len(table_oo_scaled))
print(len(table_oa_scaled))

7889
102


# Ступень 1 - NearestNeighbors

In [139]:
n_neighbors = 1  # Количество ближайших соседей
metric = 'manhattan' # Список метрик для перебора, minkowski, manhattan
algorithm = 'ball_tree' # Список алгоритмов для перебора

knn = NearestNeighbors(
                       n_neighbors = n_neighbors,
                       algorithm = algorithm,
                       metric = metric
                      )

knn.fit(table_oa_scaled)

distances, neighbors_indices = knn.kneighbors(table_oo_scaled)

In [152]:
df_sample['nn_index'] = neighbors_indices.flatten()
df_sample['dist'] = distances.flatten()

/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [141]:
df_sample['nn_index'].unique()

array([ 40,  29,  41,  39, 100,  76,  13,  83,  55,  22,  53,  59,  14,
         4,  27,  95,  57,  61,  77,  23,  42,  21,  47,  71,  49,  48,
        19,  68,  78,  75,   3,  18,  58,  84,  17,  63,  54,  92,  28,
        67,  62,  16,  87,  52,   6,  43,   5,  34,  86,  80,  37,  33,
         1,  89,  51,  82,  73,  85,  45, 101,  36,   2,  26,  31,  30,
        96,  64,  50,  46,   0,  99,  20,  79,  70,  25,  94,  88,  32,
        44,  72,  56])

In [142]:
mapping = dict(zip(df_elit.index, df_elit['Адрес_ya']))
df_sample['OA_add'] = df_sample['nn_index'].apply(lambda x: mapping.get(x))

mapping = dict(zip(df_coord_oa.index, df_coord_oa['lat']))
df_sample['lat_OA'] = df_sample['nn_index'].apply(lambda x: mapping.get(x))

mapping = dict(zip(df_coord_oa.index, df_coord_oa['lon']))
df_sample['lon_OA'] = df_sample['nn_index'].apply(lambda x: mapping.get(x))

df_sample

/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://p

,Цена(кв.м),ОПлощадь,Этаж_1,ГП,Ремонт_1,Районы_2,Сцепка_к,ЖК_БД,Терраса,Адрес_yandex,lat,lon,nn_index,OA_add,lat_OA,lon_OA
0,99573.257468,70.30,3,2009.0,стандартный,Академический,60.514731 56.810482,нет,0.0,"Екатеринбург, улица Соболева, 19",56.810482,60.514731,40,"Россия, Свердловская область, Екатеринбург, ул...",56.816075,60.593558
1,106435.643564,40.40,4,1960.0,стандартный,Центральный,60.636515 56.826183,нет,0.0,"Екатеринбург, улица Декабристов, 4",56.826183,60.636515,29,"Россия, Свердловская область, Екатеринбург, ул...",56.816956,60.634548
2,124941.065535,42.42,14,2012.0,стандартный,Пионерский,60.648157 56.864782,нет,0.0,"Екатеринбург, Боровая улица, 31",56.864782,60.648157,41,"Россия, Свердловская область, Екатеринбург, Со...",56.856924,60.641420
3,113556.338028,56.80,4,2014.0,стандартный,Уктус,60.681647 56.782174,нет,0.0,"Екатеринбург, Алтайская улица, 62",56.782174,60.681647,39,"Россия, Свердловская область, Екатеринбург, Он...",56.799926,60.637315
4,168552.036199,44.20,6,2016.0,стандартный,Южный,60.616276 56.804894,нет,0.0,"Екатеринбург, улица Степана Разина, 107Ак2",56.804894,60.616276,100,"Россия, Свердловская область, Екатеринбург, ул...",56.807422,60.614336
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7884,141451.612903,31.00,13,2017.0,стандартный,Юго_Западный,60.572447 56.81096,нет,0.0,"Екатеринбург, улица Громова, 26",56.810960,60.572447,40,"Россия, Свердловская область, Екатеринбург, ул...",56.816075,60.593558
7885,99475.065617,38.10,9,1994.0,стандартный,Уктус,60.649271 56.795727,нет,0.0,"Екатеринбург, Просторная улица, 85",56.795727,60.649271,39,"Россия, Свердловская область, Екатеринбург, Он...",56.799926,60.637315
7886,106543.209877,32.40,9,1994.0,стандартный,Уктус,60.649271 56.795727,нет,0.0,"Екатеринбург, Просторная улица, 85",56.795727,60.649271,39,"Россия, Свердловская область, Екатеринбург, Он...",56.799926,60.637315
7887,118902.439024,32.80,6,1994.0,стандартный,Уктус,60.649271 56.795727,нет,0.0,"Екатеринбург, Просторная улица, 85",56.795727,60.649271,39,"Россия, Свердловская область, Екатеринбург, Он...",56.799926,60.637315


# Ступень 2 - Haversine

In [143]:
def calculate_distance(lat1, lon1, lat2, lon2):
    # Преобразуем градусы в радианы
    lat1_rad = np.radians(lat1)
    lon1_rad = np.radians(lon1)
    lat2_rad = np.radians(lat2)
    lon2_rad = np.radians(lon2)
    
    # Создаем массив для координат
    coords_1 = np.array([[lat1_rad, lon1_rad]])
    coords_2 = np.array([[lat2_rad, lon2_rad]])
    
    # Вычисляем расстояние
    return haversine_distances(coords_1, coords_2) * 6371000  # Умножаем на радиус Земли в метрах

In [145]:
len(df_sample['nn_index'])

7889

In [148]:
metrs = []

for j in tqdm(range(len(df_sample['nn_index'])), total = df_sample.shape[0], position = 0):
    metrs.append(round(calculate_distance(df_sample['lat'].iloc[j], df_sample['lon'].iloc[j], df_sample['lat_OA'].iloc[j], df_sample['lon_OA'].iloc[j])[0][0], 0))

#metrs
df_sample['metrs'] = metrs
df_sample

100%|██████████| 7889/7889 [00:00<00:00, 9033.15it/s]
/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,Цена(кв.м),ОПлощадь,Этаж_1,ГП,Ремонт_1,Районы_2,Сцепка_к,ЖК_БД,Терраса,Адрес_yandex,lat,lon,nn_index,OA_add,lat_OA,lon_OA,metrs
0,99573.257468,70.30,3,2009.0,стандартный,Академический,60.514731 56.810482,нет,0.0,"Екатеринбург, улица Соболева, 19",56.810482,60.514731,40,"Россия, Свердловская область, Екатеринбург, ул...",56.816075,60.593558,4838.0
1,106435.643564,40.40,4,1960.0,стандартный,Центральный,60.636515 56.826183,нет,0.0,"Екатеринбург, улица Декабристов, 4",56.826183,60.636515,29,"Россия, Свердловская область, Екатеринбург, ул...",56.816956,60.634548,1033.0
2,124941.065535,42.42,14,2012.0,стандартный,Пионерский,60.648157 56.864782,нет,0.0,"Екатеринбург, Боровая улица, 31",56.864782,60.648157,41,"Россия, Свердловская область, Екатеринбург, Со...",56.856924,60.641420,965.0
3,113556.338028,56.80,4,2014.0,стандартный,Уктус,60.681647 56.782174,нет,0.0,"Екатеринбург, Алтайская улица, 62",56.782174,60.681647,39,"Россия, Свердловская область, Екатеринбург, Он...",56.799926,60.637315,3344.0
4,168552.036199,44.20,6,2016.0,стандартный,Южный,60.616276 56.804894,нет,0.0,"Екатеринбург, улица Степана Разина, 107Ак2",56.804894,60.616276,100,"Россия, Свердловская область, Екатеринбург, ул...",56.807422,60.614336,305.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7884,141451.612903,31.00,13,2017.0,стандартный,Юго_Западный,60.572447 56.81096,нет,0.0,"Екатеринбург, улица Громова, 26",56.810960,60.572447,40,"Россия, Свердловская область, Екатеринбург, ул...",56.816075,60.593558,1405.0
7885,99475.065617,38.10,9,1994.0,стандартный,Уктус,60.649271 56.795727,нет,0.0,"Екатеринбург, Просторная улица, 85",56.795727,60.649271,39,"Россия, Свердловская область, Екатеринбург, Он...",56.799926,60.637315,865.0
7886,106543.209877,32.40,9,1994.0,стандартный,Уктус,60.649271 56.795727,нет,0.0,"Екатеринбург, Просторная улица, 85",56.795727,60.649271,39,"Россия, Свердловская область, Екатеринбург, Он...",56.799926,60.637315,865.0
7887,118902.439024,32.80,6,1994.0,стандартный,Уктус,60.649271 56.795727,нет,0.0,"Екатеринбург, Просторная улица, 85",56.795727,60.649271,39,"Россия, Свердловская область, Екатеринбург, Он...",56.799926,60.637315,865.0


# Ступень 3 - Fuzz

In [149]:
sim = []

for i in tqdm(range(len(df_sample['nn_index'])), total = df_sample.shape[0], position = 0):
    token_set_ratio = fuzz.token_set_ratio(df_sample['Адрес_yandex'][i], df_sample['OA_add'][i])
    sim.append(token_set_ratio)

df_sample['sim'] = sim
df_sample

100%|██████████| 7889/7889 [00:03<00:00, 2276.44it/s]
/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,Цена(кв.м),ОПлощадь,Этаж_1,ГП,Ремонт_1,Районы_2,Сцепка_к,ЖК_БД,Терраса,Адрес_yandex,lat,lon,nn_index,OA_add,lat_OA,lon_OA,metrs,sim
0,99573.257468,70.30,3,2009.0,стандартный,Академический,60.514731 56.810482,нет,0.0,"Екатеринбург, улица Соболева, 19",56.810482,60.514731,40,"Россия, Свердловская область, Екатеринбург, ул...",56.816075,60.593558,4838.0,75
1,106435.643564,40.40,4,1960.0,стандартный,Центральный,60.636515 56.826183,нет,0.0,"Екатеринбург, улица Декабристов, 4",56.826183,60.636515,29,"Россия, Свердловская область, Екатеринбург, ул...",56.816956,60.634548,1033.0,72
2,124941.065535,42.42,14,2012.0,стандартный,Пионерский,60.648157 56.864782,нет,0.0,"Екатеринбург, Боровая улица, 31",56.864782,60.648157,41,"Россия, Свердловская область, Екатеринбург, Со...",56.856924,60.641420,965.0,77
3,113556.338028,56.80,4,2014.0,стандартный,Уктус,60.681647 56.782174,нет,0.0,"Екатеринбург, Алтайская улица, 62",56.782174,60.681647,39,"Россия, Свердловская область, Екатеринбург, Он...",56.799926,60.637315,3344.0,73
4,168552.036199,44.20,6,2016.0,стандартный,Южный,60.616276 56.804894,нет,0.0,"Екатеринбург, улица Степана Разина, 107Ак2",56.804894,60.616276,100,"Россия, Свердловская область, Екатеринбург, ул...",56.807422,60.614336,305.0,62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7884,141451.612903,31.00,13,2017.0,стандартный,Юго_Западный,60.572447 56.81096,нет,0.0,"Екатеринбург, улица Громова, 26",56.810960,60.572447,40,"Россия, Свердловская область, Екатеринбург, ул...",56.816075,60.593558,1405.0,77
7885,99475.065617,38.10,9,1994.0,стандартный,Уктус,60.649271 56.795727,нет,0.0,"Екатеринбург, Просторная улица, 85",56.795727,60.649271,39,"Россия, Свердловская область, Екатеринбург, Он...",56.799926,60.637315,865.0,72
7886,106543.209877,32.40,9,1994.0,стандартный,Уктус,60.649271 56.795727,нет,0.0,"Екатеринбург, Просторная улица, 85",56.795727,60.649271,39,"Россия, Свердловская область, Екатеринбург, Он...",56.799926,60.637315,865.0,72
7887,118902.439024,32.80,6,1994.0,стандартный,Уктус,60.649271 56.795727,нет,0.0,"Екатеринбург, Просторная улица, 85",56.795727,60.649271,39,"Россия, Свердловская область, Екатеринбург, Он...",56.799926,60.637315,865.0,72


In [153]:
s_1 = df_sample['metrs'] <= 30
s_2 = df_sample['sim'] >= 94

df_sample[s_1 & s_2]

,Цена(кв.м),ОПлощадь,Этаж_1,ГП,Ремонт_1,Районы_2,Сцепка_к,ЖК_БД,Терраса,Адрес_yandex,lat,lon,nn_index,OA_add,lat_OA,lon_OA,metrs,sim,dist
8,197029.702970,50.5,5,2021.0,стандартный,Южный,60.593962 56.805116,нет,0.0,"Екатеринбург, Московская улица, 249",56.805116,60.593962,13,"Россия, Свердловская область, Екатеринбург, Мо...",56.805116,60.593962,0.0,100,0.0
70,181558.441558,38.5,9,2023.0,требует_косметического_ремонта,Южный,60.637315 56.799926,нет,0.0,"Екатеринбург, Онежская улица, 4",56.799926,60.637315,39,"Россия, Свердловская область, Екатеринбург, Он...",56.799926,60.637315,0.0,100,0.0
73,181216.931217,75.6,10,2023.0,требует_косметического_ремонта,Южный,60.637315 56.799926,нет,0.0,"Екатеринбург, Онежская улица, 4",56.799926,60.637315,39,"Россия, Свердловская область, Екатеринбург, Он...",56.799926,60.637315,0.0,100,0.0
97,266798.418972,50.6,6,2022.0,требует_косметического_ремонта,Вокзальный,60.589515 56.851845,престижный,0.0,"Екатеринбург, Печёрская улица, 4",56.851845,60.589515,61,"Россия, Свердловская область, Екатеринбург, Пе...",56.851845,60.589515,0.0,100,0.0
155,226843.100189,52.9,11,2018.0,улучшенный,Южный,60.614336 56.807422,нет,0.0,"Екатеринбург, улица Щорса, 53",56.807422,60.614336,100,"Россия, Свердловская область, Екатеринбург, ул...",56.807422,60.614336,0.0,100,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7774,365384.615385,52.0,14,2018.0,премиальный,Клевер_Парк,60.634548 56.816956,престижный,0.0,"Екатеринбург, улица Ткачей, 13",56.816956,60.634548,29,"Россия, Свердловская область, Екатеринбург, ул...",56.816956,60.634548,0.0,100,0.0
7779,231038.961039,46.2,7,2020.0,требует_косметического_ремонта,Клевер_Парк,60.629769 56.815764,престижный,0.0,"Екатеринбург, Машинная улица, 1В",56.815764,60.629769,34,"Россия, Свердловская область, Екатеринбург, Ма...",56.815764,60.629769,0.0,100,0.0
7807,225000.000000,82.0,16,2024.0,требует_косметического_ремонта,Центральный,60.618899 56.825006,престижный,0.0,"Екатеринбург, улица Декабристов, 20",56.825006,60.618899,37,"Россия, Свердловская область, Екатеринбург, ул...",56.825006,60.618899,0.0,100,0.0
7815,165259.740260,123.2,11,2022.0,требует_косметического_ремонта,Вокзальный,60.589515 56.851845,престижный,0.0,"Екатеринбург, Печёрская улица, 4",56.851845,60.589515,61,"Россия, Свердловская область, Екатеринбург, Пе...",56.851845,60.589515,0.0,100,0.0


# Ступень 4 - Mapping

In [160]:
s_1 = df_sample['metrs'] <= 30
s_2 = df_sample['sim'] >= 94

df_map_el = df_sample[s_1 & s_2]
df_map_el.reset_index(drop = True , inplace = True)

mapping = dict(zip(df_map_el['Сцепка_к'], ['elit'] * len(df_map_el['Сцепка_к'])))
df_sample['is_elit'] = df_sample['Сцепка_к'].apply(lambda x: mapping.get(x))

df_sample['is_elit'] = np.where(df_sample['is_elit'].isnull(), 'no', df_sample['is_elit'])

df_sample

/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/python37/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,Цена(кв.м),ОПлощадь,Этаж_1,ГП,Ремонт_1,Районы_2,Сцепка_к,ЖК_БД,Терраса,Адрес_yandex,lat,lon,nn_index,OA_add,lat_OA,lon_OA,metrs,sim,dist,is_elit
0,99573.257468,70.30,3,2009.0,стандартный,Академический,60.514731 56.810482,нет,0.0,"Екатеринбург, улица Соболева, 19",56.810482,60.514731,40,"Россия, Свердловская область, Екатеринбург, ул...",56.816075,60.593558,4838.0,75,0.248409,no
1,106435.643564,40.40,4,1960.0,стандартный,Центральный,60.636515 56.826183,нет,0.0,"Екатеринбург, улица Декабристов, 4",56.826183,60.636515,29,"Россия, Свердловская область, Екатеринбург, ул...",56.816956,60.634548,1033.0,72,0.049081,no
2,124941.065535,42.42,14,2012.0,стандартный,Пионерский,60.648157 56.864782,нет,0.0,"Екатеринбург, Боровая улица, 31",56.864782,60.648157,41,"Россия, Свердловская область, Екатеринбург, Со...",56.856924,60.641420,965.0,77,0.056055,no
3,113556.338028,56.80,4,2014.0,стандартный,Уктус,60.681647 56.782174,нет,0.0,"Екатеринбург, Алтайская улица, 62",56.782174,60.681647,39,"Россия, Свердловская область, Екатеринбург, Он...",56.799926,60.637315,3344.0,73,0.208630,no
4,168552.036199,44.20,6,2016.0,стандартный,Южный,60.616276 56.804894,нет,0.0,"Екатеринбург, улица Степана Разина, 107Ак2",56.804894,60.616276,100,"Россия, Свердловская область, Екатеринбург, ул...",56.807422,60.614336,305.0,62,0.017393,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7884,141451.612903,31.00,13,2017.0,стандартный,Юго_Западный,60.572447 56.81096,нет,0.0,"Екатеринбург, улица Громова, 26",56.810960,60.572447,40,"Россия, Свердловская область, Екатеринбург, ул...",56.816075,60.593558,1405.0,77,0.083596,no
7885,99475.065617,38.10,9,1994.0,стандартный,Уктус,60.649271 56.795727,нет,0.0,"Екатеринбург, Просторная улица, 85",56.795727,60.649271,39,"Россия, Свердловская область, Екатеринбург, Он...",56.799926,60.637315,865.0,72,0.053488,no
7886,106543.209877,32.40,9,1994.0,стандартный,Уктус,60.649271 56.795727,нет,0.0,"Екатеринбург, Просторная улица, 85",56.795727,60.649271,39,"Россия, Свердловская область, Екатеринбург, Он...",56.799926,60.637315,865.0,72,0.053488,no
7887,118902.439024,32.80,6,1994.0,стандартный,Уктус,60.649271 56.795727,нет,0.0,"Екатеринбург, Просторная улица, 85",56.795727,60.649271,39,"Россия, Свердловская область, Екатеринбург, Он...",56.799926,60.637315,865.0,72,0.053488,no


In [162]:
df_sample['is_elit'].value_counts()

no      7609
elit     280
Name: is_elit, dtype: int64